In [1]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema
from coffea.analysis_tools import PackedSelection
import hist
from coffea import processor
import awkward as ak
import numba
import vector
from collections import namedtuple
vector.register_awkward()

In [2]:
events,report = NanoEventsFactory.from_root(
    '../data/p8_ee_ZH_ecm240/events_082532938.root:events',
    schemaclass=BaseSchema,
    metadata={'dataset':'ZH'},
    uproot_options={"allow_read_errors_with_report": True}
).events()

In [6]:
report.compute()

<Array [{call_time: None, duration: 0.18, ...}] type='1 * {call_time: ?unkn...'>

In [3]:
events.fields

['Electron',
 'Electron#0',
 'Electron#0/Electron#0.index',
 'Electron#0/Electron#0.collectionID',
 'Muon',
 'Muon#0',
 'Muon#0/Muon#0.index',
 'Muon#0/Muon#0.collectionID',
 'AllMuon',
 'AllMuon#0',
 'AllMuon#0/AllMuon#0.index',
 'AllMuon#0/AllMuon#0.collectionID',
 'EFlowNeutralHadron',
 'EFlowNeutralHadron/EFlowNeutralHadron.type',
 'EFlowNeutralHadron/EFlowNeutralHadron.energy',
 'EFlowNeutralHadron/EFlowNeutralHadron.energyError',
 'EFlowNeutralHadron/EFlowNeutralHadron.position.x',
 'EFlowNeutralHadron/EFlowNeutralHadron.position.y',
 'EFlowNeutralHadron/EFlowNeutralHadron.position.z',
 'EFlowNeutralHadron/EFlowNeutralHadron.positionError[6]',
 'EFlowNeutralHadron/EFlowNeutralHadron.iTheta',
 'EFlowNeutralHadron/EFlowNeutralHadron.phi',
 'EFlowNeutralHadron/EFlowNeutralHadron.directionError.x',
 'EFlowNeutralHadron/EFlowNeutralHadron.directionError.y',
 'EFlowNeutralHadron/EFlowNeutralHadron.directionError.z',
 'EFlowNeutralHadron/EFlowNeutralHadron.shapeParameters_begin',
 'EFlo

In [4]:
def index_mask(input_array, index_array):
    '''
    This function matches the given attribute of ReconstructedParticles (for example energy) to the particle index (for example Muon or Electron).
    Note: Input and output are both awkward.Array; dask_awkward is not supported at this moment 
    '''
    if len(input_array) != len(index_array) :
        raise Exception(f'Length of input_array({len(input_array)}) and index_array({len(index_array)}) does not match!')
    counts = len(input_array)

    @numba.jit
    def numba_wrap(input_array, index_array,counts):
        output_array = []
        for event_index in range(counts):
            event_mask = index_array[event_index]
            reco_list = input_array[event_index]
            output_array.append([reco_list[i] for i in  event_mask])
        return output_array
    out = ak.Array(numba_wrap(input_array,index_array,counts))

    return out
    
def get(events,collection,attribute):
    return events[collection+'/'+collection+'.'+attribute].compute()
    
def get_with_cut(events,collection,attribute,cut):
    return ak.mask(events[collection+'/'+collection+'.'+attribute].compute(),cut)
    
def get_reco(Reconstr_branch, needed_particle):
    part = namedtuple('particle', list(Reconstr_branch._fields))
    return part(*[index_mask(getattr(Reconstr_branch,attr), get(events,needed_particle,'index')) for attr in Reconstr_branch._fields])

def get_all(events,Collection,cut):
    prefix = '/'.join([Collection]*2)+'.'
    list_of_attr = [field.replace(prefix,'') for field in events.fields if field.startswith(prefix)]
    replace_list = ['.','[',']']
    valid_attr = list_of_attr
    for rep in replace_list:
        valid_attr = [field.replace(rep, '_') for field in valid_attr ]
    part = namedtuple('particle', valid_attr)
    return part(*[get_with_cut(events,Collection,attr,cut) for attr in list_of_attr])

In [5]:
# Create a Packed Selection object to get a cutflow later
cut = PackedSelection()
Recon = get(events,'ReconstructedParticles','energy')
cut.add('No cut', ak.all(Recon > 0, axis=1))

# Filter out any event with no reconstructed particles and generate Reconstructed Particle Attributes
#ak.mask preserves array length
Reco = get_all(events,'ReconstructedParticles', ak.all(Recon > 0, axis=1))
cut.add('At least one Reco Particle', ak.all(Reco.energy > 0, axis=1))

# Generate Muon Attributes
Muon = get_reco(Reco,'Muon#0')

In [6]:
Muon.energy

<Array [[], [], [], [75.9], ..., [], [], [], []] type='100000 * var * float64'>

In [7]:
Muon._fields

('type',
 'energy',
 'momentum_x',
 'momentum_y',
 'momentum_z',
 'referencePoint_x',
 'referencePoint_y',
 'referencePoint_z',
 'charge',
 'mass',
 'goodnessOfPID',
 'covMatrix_10_',
 'clusters_begin',
 'clusters_end',
 'tracks_begin',
 'tracks_end',
 'particles_begin',
 'particles_end',
 'particleIDs_begin',
 'particleIDs_end')